In [5]:
import sys
sys.path.append('../')

%load_ext autoreload
%autoreload 2

import os
from custom_datasets.rollout_push_any import RolloutPushAnyMemDataset
from custom_datasets.pusht import PushTDataset
from concurrent.futures import ThreadPoolExecutor, wait

import re
from pathlib import Path
import zarr
import numpy as np
import torch
import torch.nn.functional as F
import imageio
from IPython.display import Video

from typing import Dict
import time
from custom_datasets.rollout_push_any import RolloutPushAnyMemDataset
from custom_datasets.core import TrajectoryEmbeddingDataset, split_traj_datasets
from custom_datasets.vqbet_repro import TrajectorySlicerDataset

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
device = 'cuda'

In [46]:
checkpoint_root_path = '/home/sm/Datasets/dynamo/experiments/checkpoints/pusht'
exp_path = checkpoint_root_path + '/2024-11-21/12-17-14/playful-morning-36'
pretrain_exp_path = '/home/sm/PycharmProjects/dynamo_ssl/exp_local/for_examination/013554_train_pusht_rollout_dynamo'
encoder_path = pretrain_exp_path + "/encoder_59.pt"
check_point = exp_path + '/completion_idx_3100.json'

org_encoder_path = '/home/sm/PycharmProjects/dynamo_ssl/exp_local/2024.11.12/141256_train_pusht_dynamo/encoder.pt'

# Encoder Inspection

## Create Encoder

In [14]:
encoder = torch.load("/home/sm/PycharmProjects/dynamo_ssl/exp_local/for_examination/013554_train_pusht_rollout_dynamo/encoder_59.pt")
encoder = encoder.to(device).eval()

In [47]:
org_encoder = torch.load(org_encoder_path)
org_encoder = org_encoder.to(device).eval()

In [48]:
rollout_dataset = RolloutPushAnyMemDataset('/home/sm/Datasets/lerobot/pusht')
pusht_dataset = PushTDataset('/home/sm/Datasets/dynamo/datasets/dynamo_repro_datasets/pusht_dataset')

In [49]:
pusht_embedding_dataset = TrajectoryEmbeddingDataset(
    org_encoder, pusht_dataset, device=device, embed_goal=False
)

In [15]:
# embedding_dataset = TrajectoryEmbeddingDataset(
#     encoder, rollout_dataset, device=device, embed_goal=False
# )

embedding_dataset = torch.load(pretrain_exp_path + '/embedding.pth')

In [57]:
def anaylze_variances(embedding_dataset, num_episodes=100):
    num_episodes = 100
    episode_indices = np.random.randint(0, len(embedding_dataset), size=num_episodes)
    features_of_episodes = [embedding_dataset[i][0].squeeze(1) for i in episode_indices]
    
    features = torch.cat(features_of_episodes, dim=0)
    features = features.cpu().numpy()
    dim_variances = np.var(features, axis=0)
    img_variances = np.var(features, axis=1)
    print("Internal Variance")
    print_variance(dim_variances)
    print()
    print("Inter Variance")
    print_variance(img_variances)

In [60]:
def print_variance(variances):
    print("Mean Variance:", np.mean(variances))
    for threshold in [1e-5, 1e-4, 1e-3, 1e-2, 1e-1]:
        low_variance_ratio = np.sum(variances < threshold) / len(variances)
        print(f"Low variance dimensions ratio (< {threshold}): {low_variance_ratio:.2%}")

In [61]:
anaylze_variances(embedding_dataset)

Internal Variance
Mean Variance: 0.021553116
Low variance dimensions ratio (< 1e-05): 19.34%
Low variance dimensions ratio (< 0.0001): 33.20%
Low variance dimensions ratio (< 0.001): 54.88%
Low variance dimensions ratio (< 0.01): 83.20%
Low variance dimensions ratio (< 0.1): 94.92%

Inter Variance
Mean Variance: 0.05111394
Low variance dimensions ratio (< 1e-05): 0.00%
Low variance dimensions ratio (< 0.0001): 0.00%
Low variance dimensions ratio (< 0.001): 0.00%
Low variance dimensions ratio (< 0.01): 0.00%
Low variance dimensions ratio (< 0.1): 97.32%


In [62]:
anaylze_variances(pusht_embedding_dataset)

Internal Variance
Mean Variance: 0.027900876
Low variance dimensions ratio (< 1e-05): 0.00%
Low variance dimensions ratio (< 0.0001): 0.00%
Low variance dimensions ratio (< 0.001): 0.00%
Low variance dimensions ratio (< 0.01): 16.02%
Low variance dimensions ratio (< 0.1): 97.85%

Inter Variance
Mean Variance: 0.037196297
Low variance dimensions ratio (< 1e-05): 0.00%
Low variance dimensions ratio (< 0.0001): 0.00%
Low variance dimensions ratio (< 0.001): 0.00%
Low variance dimensions ratio (< 0.01): 0.00%
Low variance dimensions ratio (< 0.1): 99.87%
